## Study Caffe 01 : 2019/02/23

> blog 1: [Deep learning tutorial on Caffe technology : basic commands, Python and C++ code](http://christopher5106.github.io/deep/learning/2015/09/04/Deep-learning-tutorial-on-Caffe-Technology.html);  
> blog 2: [A Practical Introduction to Deep Learning with Caffe and Python](http://adilmoujahid.com/posts/2016/06/introduction-deep-learning-python-caffe/)

## Define a network model
Let’s create first a very simple model with a single convolution composed of 3 convolutional neurons, with kernel of size 5x5 and stride of 1.  
![exam1](http://christopher5106.github.io/img/simple_network.png)  

This net will produce 3 output maps from an input map.

The output map for a convolution given receptive field size has a dimension given by the following equation :  
`output = (input - kernel_size) / stride + 1`

Create a first file `conv.prototxt` describing the neuron network :

```
name: "convolution"
input: "data"
input_dim: 1
input_dim: 100
layer {
  name: "conv"
  type: "Convolution"
  bottom: "data"
  top: "conv"
  convolution_param {
    num_output: 3
    kernel_size: 5
    stride: 1
    weight_filler {
      type: "gaussian"
      std: 0.01
    }
    bias_filler {
      type: "constant"
      value: 0
    }
  }
}
```
with one layer, a convolution, from the [Catalog of available layers](http://caffe.berkeleyvision.org/tutorial/layers.html)

Load the net via  
`net = caffe.Net('conv.prototxt', caffe.TEST)`  

The names of input layers of the net are given by `print net.inputs`.

The net contains two ordered dictionaries:

- `net.blobs` for input data and its propagation in the layers:  
`net.blobs['data']` contains input data, an array of shape (1, 1, 100, 100), `net.blobs['conv']` contains computed data in layer ‘conv’ (1, 3, 96, 96) initialiazed with zeros.

To print the infos, `[(k, v.data.shape) for k, v in net.blobs.items()]`

- `net.params` a vector of blobs for weight and bias parameters:  
`net.params['conv'][0]` contains the weight parameters, an array of shape (3, 1, 5, 5)  
`net.params['conv'][1]` contains the bias parameters, an array of shape (3,)  
initialiazed with ‘weight_filler’ and ‘bias_filler’ algorithms.  
To print the infos : `[(k, v[0].data.shape, v[1].data.shape) for k, v in net.params.items()]`

Blobs are memory abstraction objects (with execution depending on the mode), and data is contained in the field data as an array:

> print net.blobs['conv'].data.shape

To draw the network, a simle python command:  
> python python/draw_net.py examples/net_surgery/conv.prototxt my_net.png
> open my_net.png

This will print the following image:


## Compute the net output on an image as input
Let’s load a gray image of size 1x360x480 (channel x height x width) into the previous net:  
![cat](http://christopher5106.github.io/img/cat_gray.jpg)

We need to reshape the data blob (1, 1, 100, 100) to the new size (1, 1, 360, 480) to fit the image:

```python
im = np.array(Image.open('examples/images/cat_gray.jpg'))
im_input = im[np.newaxis, np.newaxis, :, :]
net.blobs['data'].reshape(*im_input.shape)
net.blobs['data'].data[...] = im_input
```
Let’s compute the blobs given this input
```python
net.forward()
```
Now `net.blobs['conv']` is filled with data, and the 3 pictures inside each of the 3 neurons (`net.blobs['conv'].data[0,i]`) can be plotted easily.

To save the net parameters `net.params`, just call :
```python
net.save('mymodel.caffemodel')
```

## Load pretrained parameters to classify an image

In the previous net, weight and bias params have been initialiazed randomly.

It is possible to load trained parameters and in this case, the result of the net will produce a classification.

Many trained models can be downloaded from the community in the `Caffe Model Zoo`, such as car classification, flower classification, digit classification…

Model informations are written in Github Gist format. The parameters are saved in a `.caffemodel` file specified in the gist. To download the model :

```
./scripts/download_model_from_gist.sh <gist_id>
./scripts/download_model_binary.py <dirname>
```

where is the gist directory (by default the gist is saved in the **models** directory).

Let’s download the CaffeNet model and the labels corresponding to the classes:

```
./scripts/download_model_binary.py models/bvlc_reference_caffenet
./data/ilsvrc12/get_ilsvrc_aux.sh

#have a look at the model
python python/draw_net.py models/bvlc_reference_caffenet/deploy.prototxt caffenet.png
open caffenet.png
```


This model has been trained on processed images, so you need to preprocess the image with a preprocessor, before saving it in the blob.

That is, in the python shell:
```python
#load the model
net = caffe.Net('models/bvlc_reference_caffenet/deploy.prototxt',
                'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
                caffe.TEST)

# load input and configure preprocessing
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_mean('data', np.load('python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1))
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

#note we can change the batch size on-the-fly
#since we classify only one image, we change batch size from 10 to 1
net.blobs['data'].reshape(1,3,227,227)

#load the image in the data layer
im = caffe.io.load_image('examples/images/cat.jpg')
net.blobs['data'].data[...] = transformer.preprocess('data', im)

#compute
out = net.forward()

# other possibility : out = net.forward_all(data=np.asarray([transformer.preprocess('data', im)]))

#predicted predicted class
print out['prob'].argmax()

#print predicted labels
labels = np.loadtxt("data/ilsvrc12/synset_words.txt", str, delimiter='\t')
top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
print labels[top_k]
```
It will print you the top classes detected for the images.

Go further : Create a classification map with net surgery to insert a trained model into an extended model where convolutions will be innerproducts spatially

## Learn : solve the params on training data
It is now time to create your own model, and training the parameters on training data.

To train a network, you need

its model definition, as seen previously

a second protobuf file, the solver file, describing the parameters for the stochastic gradient.

For example, the CaffeNet solver :

```
net: "models/bvlc_reference_caffenet/train_val.prototxt"
test_iter: 1000
test_interval: 1000
base_lr: 0.01
lr_policy: "step"
gamma: 0.1
stepsize: 100000
display: 20
max_iter: 450000
momentum: 0.9
weight_decay: 0.0005
snapshot: 10000
snapshot_prefix: "models/bvlc_reference_caffenet/caffenet_train"
solver_mode: GPU
```

Usually, you define a train net, for training, with training data, and a test set, for validation. Either you can define the train and test nets in the prototxt solver file:
```
train_net: "examples/hdf5_classification/nonlinear_auto_train.prototxt"
test_net: "examples/hdf5_classification/nonlinear_auto_test.prototxt"
```
or you can also specify only one prototxt file, adding an include phase statement for the layers that have to be different in training and testing phases, such as input data :

```
layer {
  name: "data"
  type: "Data"
  top: "data"
  top: "label"
  include {
    phase: TRAIN
  }
  data_param {
    source: "examples/imagenet/ilsvrc12_train_lmdb"
    batch_size: 256
    backend: LMDB
  }
}
layer {
  name: "data"
  type: "Data"
  top: "data"
  top: "label"
  top: "label"
  include {
    phase: TEST
  }
  data_param {
    source: "examples/imagenet/ilsvrc12_val_lmdb"
    batch_size: 50
    backend: LMDB
  }
}
```


## To be continued ... 